In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## funzione BALL()

In [2]:
function ball(radius=1, angle1=pi, angle2=2*pi)
    function ball0(shape=[18, 36, 4])
        V, CV = Lar.cuboidGrid(shape)
        V = [angle1/shape[1] 0 0; 0 angle2/shape[2] 0; 0 0 radius/shape[3]]*V
        V = broadcast(+, V, [-(angle1)/2, -(angle2)/2, 0])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let
        (u, v, w)=p; [w*cos(u)*cos(v); w*cos(u)*sin(v); w*sin(u)] end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return ball0
end


ball (generic function with 4 methods)

In [3]:
@btime W,CW = Lar.ball()();

  20.399 ms (277370 allocations: 21.92 MiB)


In [4]:
@code_warntype Lar.ball()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#ball0#312"{Int64,Irrational{:π},Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(18, 36, 4)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


funzione NON è type unstable poiche Body::Tuple{Array,Array{Array{Int64,1},1}}


In [5]:
@benchmark Lar.ball()()

BenchmarkTools.Trial: 200 samples with 1 evaluation.
 Range (min … max):  20.504 ms … 39.432 ms  ┊ GC (min … max): 0.00% … 12.24%
 Time  (median):     24.389 ms              ┊ GC (median):    6.79%
 Time  (mean ± σ):   25.029 ms ±  3.603 ms  ┊ GC (mean ± σ):  8.22% ±  7.73%

  █             ▄       ▁                                      
  ██▅▆▄▄▃▃▄▆▆▆▃▅█▇▇▃▄▄▄▃█▆▄▃▄▄▃▆▃▃▁▅▃▃▄▄▃▃▃▄▁▁▃▃▄▃▁▃▁▁▁▃▁▁▁▁▃ ▃
  20.5 ms         Histogram: frequency by time        35.5 ms <

 Memory estimate: 21.92 MiB, allocs estimate: 277368.

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [6]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [11]:
function ball2(radius=1, angle1=pi, angle2=2*pi)
    function ball02(shape=[18, 36, 4])
        V, CV = Lar.cuboidGrid(shape)
        V = [angle1/shape[1] 0 0; 0 angle2/shape[2] 0; 0 0 radius/shape[3]]*V
        V = broadcast(+, V, [-(angle1)/2, -(angle2)/2, 0])
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
        end 
        V = hcat(map(p->let
        (u, v, w)=p; [w*cos(u)*cos(v); w*cos(u)*sin(v); w*sin(u)] end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return ball02
end

ball2 (generic function with 4 methods)

In [12]:
@btime W,CW = ball2()();

  17.564 ms (241358 allocations: 19.01 MiB)


Tempo esecuzione migliorato di poco , uso dei threads applicabile solo in questo modo , il ciclo for è stato esternato (obbligatorio infatti anche nelle altre funzioni è stato fatto cosi ) per permettermi di usare la macro @threads . @inbounds spiegata nel dettaglio nella relazione.